In [1]:
import pandas as pd
from google.cloud import bigquery
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from google.cloud import storage
from tensorflow.keras import callbacks
from tensorflow.keras import Sequential, Input, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

2025-06-05 17:54:33.610351: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-05 17:54:33.627493: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-05 17:54:33.753123: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-05 17:54:33.864596: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749142473.952512  126217 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749142473.97

In [2]:
import sys
import os

# Add parent directory to Python path
sys.path.append(os.path.abspath(".."))

# Now you can import the module
import preprocessing
from data import get_metadata_from_bq

In [3]:
metadata = get_metadata_from_bq()

In [4]:
metadata.shape

(10015, 8)

In [5]:
metadata.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,index
0,HAM_0006706,ISIC_0031725,nv,histo,NaN,female,abdomen,8410
1,HAM_0002610,ISIC_0026876,vasc,consensus,0.0,female,abdomen,2388
2,HAM_0002610,ISIC_0030606,vasc,consensus,0.0,female,abdomen,2389
3,HAM_0003872,ISIC_0028903,nv,consensus,0.0,female,abdomen,9329
4,HAM_0002034,ISIC_0027954,bkl,consensus,5.0,male,abdomen,1035


In [6]:
processed_metadata = preprocessing.preprocess_metadata(metadata)

In [7]:
processed_metadata.shape

(9771, 6)

In [8]:
images_df = pd.read_json("../raw_data/resized_images.json")

In [9]:
images_df.head()

,image_id,resized_array
0,ISIC_0024306,"[[[228, 131, 133], [230, 130, 137], [231, 124,..."
1,ISIC_0024307,"[[[216, 121, 133], [218, 122, 137], [218, 125,..."
2,ISIC_0024308,"[[[246, 173, 181], [245, 171, 175], [247, 171,..."
3,ISIC_0024309,"[[[206, 118, 134], [209, 120, 136], [210, 120,..."
4,ISIC_0024310,"[[[72, 33, 38], [70, 32, 38], [69, 31, 38], [6..."


In [10]:
np.array(images_df.resized_array[0]).shape

(96, 96, 3)

In [ ]:
images_df["np_array"] = images_df.resized_array.apply(np.array)

In [ ]:
test_df = pd.DataFrame({
    'image_id': image_name,
    'resized_array': resized_array
})

In [ ]:
test_df.head()

In [ ]:
full_df = metadata.merge(test_df,how="left",on="image_id")
full_df.head()

In [ ]:
full_df.drop(columns=[col for col in ["image_id"] if col in full_df.columns],inplace=True)

In [ ]:
full_df.head()

In [ ]:
X = full_df["resized_array"]

In [ ]:
y = full_df["target"]

In [ ]:
X_np = np.array(X)

In [ ]:
X_np.shape

In [ ]:
X_np[0].shape

In [ ]:
X_ready = []
for row in X_np:
    X_ready.append(X[0])

In [ ]:
X_ready = np.array(X_ready)

In [ ]:
X_ready

In [ ]:
y.shape

In [ ]:
from tensorflow.keras.utils import to_categorical
y_cat = to_categorical(y)
y_cat

In [ ]:
X_ready.shape

In [ ]:
X_train, X_test,y_train, y_test = train_test_split(X_ready, y_cat, test_size=0.3, random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_train = X_train/255

In [ ]:
X_test = X_test/255

# Model Architecture

In [ ]:
model = Sequential()

model.add(Input(shape=(96, 96, 3)))
model.add(layers.Conv2D(16, (6, 6), padding='same', activation="relu"))
model.add(layers.Conv2D(16, (4, 4), padding='same', activation="relu"))
model.add(layers.Conv2D(32, (3, 3), padding='same', activation="relu"))
model.add(layers.Flatten())
model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=["accuracy","recall"])

In [ ]:
es = EarlyStopping(patience=10, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train,
          epochs=100,  # Use early stopping in practice
          batch_size=32,
          verbose=1,
          validation_split = 0.2,
          callbacks=[es])